In [1]:
import os

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
x_train=mnist.train.images[0:1000].reshape(1000,28,28,1)
y_train=(mnist.train.labels[0:1000])

/home/aman/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
x = tf.placeholder(tf.float32, [None, 28,28,1])
x=tf.reshape(x,[-1,28,28,1])
W_conv1=tf.Variable(tf.truncated_normal([5, 5,1,32],stddev=0.1))
b_conv1=tf.Variable(tf.truncated_normal([1,1,1,32],stddev=0.1))

z_conv_1=tf.nn.conv2d(x_train, W_conv1, strides=[1, 1, 1, 1], padding='SAME')+b_conv1
h_conv_1 = tf.nn.relu(z_conv_1)
h_pool_1=tf.nn.max_pool(h_conv_1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
fc_0=tf.reshape(h_pool_1,[-1,14*14*32])

W_fc1 = tf.Variable(tf.truncated_normal([14*14*32,1024],stddev=0.1))
b_fc1 = tf.Variable(tf.truncated_normal([1,1024],stddev=0.1))

h_fc1 = tf.nn.relu(tf.matmul(fc_0, W_fc1) + b_fc1)

W_fc2 = tf.Variable(tf.truncated_normal([1024, 10],stddev=0.1))
b_fc2 = tf.Variable(tf.truncated_normal([1,10],stddev=0.1))

y_pred = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

y = tf.placeholder(tf.float32, [None, 10])

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_train,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

saver = tf.train.Saver()

sess=tf.Session()
sess.run(tf.global_variables_initializer())
writer=tf.summary.FileWriter("./graphs", sess.graph)

In [4]:
if os.path.exists("/home/aman/Tensorflow/checkpoints/model.ckpt.meta"):
    try:
        saver.restore(sess, "/home/aman/Tensorflow/checkpoints/model.ckpt")
        print("Model restored.")
    except:
        print("Model has already been loaded once")

INFO:tensorflow:Restoring parameters from /home/aman/Tensorflow/checkpoints/model.ckpt
Model restored.


In [5]:
for i in range(10):
    print("Cross-entropy error is {} and training accuracy is {}".format(sess.run(cross_entropy,feed_dict={x:x_train, y: y_train}),sess.run(accuracy,feed_dict={x:x_train, y: y_train})))
    sess.run(train_step,feed_dict={x: x_train, y: y_train})
    
save_path = saver.save(sess, "/home/aman/Tensorflow/checkpoints/model.ckpt")
print("Model saved in path: %s" % save_path)

Cross-entropy error is 0.354853749275 and training accuracy is 0.917999982834
Cross-entropy error is 0.344523340464 and training accuracy is 0.919000029564
Cross-entropy error is 0.334428429604 and training accuracy is 0.92199999094
Cross-entropy error is 0.32481738925 and training accuracy is 0.921000003815
Cross-entropy error is 0.31581890583 and training accuracy is 0.924000024796
Cross-entropy error is 0.307426959276 and training accuracy is 0.925999999046
Cross-entropy error is 0.299539238214 and training accuracy is 0.926999986172
Cross-entropy error is 0.292089730501 and training accuracy is 0.927999973297
Cross-entropy error is 0.285036623478 and training accuracy is 0.927999973297
Cross-entropy error is 0.278291255236 and training accuracy is 0.927999973297
Model saved in path: /home/aman/Tensorflow/checkpoints/model.ckpt


In [ ]:
sess.close()
writer.close()